In [89]:
import scipy.io
import pandas as pd
import numpy as np
from itertools import chain
#import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs
from plotly.graph_objs import Bar
import plotly.plotly as py
init_notebook_mode(connected=True)
import cufflinks as cf
from ipywidgets import *

In [90]:
# load data
f_a = scipy.io.loadmat('../data/anova_mat.mat')
n_units = f_a['anova_cell'].shape[0]
infomat = f_a['infomat']
p_vals = np.zeros((n_units, 7))
p_vals[:,:4] = infomat[:,:4].astype(int)
locs = infomat[:,9]
# loop through all units 
for unit in range(n_units):
    frame_dict = {'y': np.array(list(chain.from_iterable(f_a['anova_cell'][unit][0]))), # firing rate 
                 'f1': np.array(list(chain.from_iterable(f_a['anova_cell'][unit][1]))), # mem
                 'f2': np.array(list(chain.from_iterable(f_a['anova_cell'][unit][2])))} # stimulus id
    data = pd.DataFrame(data=frame_dict, index = range(len(frame_dict['y'])))
    #print(type(data))
    
    fr_lm = ols(formula='y ~ C(f1)*C(f2)', data=data).fit()
    #print (fr_lm.summary())
    aov_table = sm.stats.anova_lm(fr_lm, typ=2)
    aov_table = aov_table.rename(columns={"PR(>F)": "p"})      
   # print(aov_table)
    
    #aov_table = anova_2f(data)
   # print(aov_table)
    p_vals[unit, 4] = aov_table.p[0] # success of encoding
    p_vals[unit, 5] = aov_table.p[1] # simulus ID
    p_vals[unit, 6] = aov_table.p[2] # interaction
        
    resFrame = pd.DataFrame(p_vals, columns = ['patient', 'session', 'channel', 'cluster', 'p_mem', 'p_stim', 'p_interaction'])
    
    #p_vals[unit, 4] = aov_table['p'][0] # success of encoding
    #p_vals[unit, 5] = aov_table['p'][1] # simulus ID
    #p_vals[unit, 6] = aov_table['p'][2] # interaction
        



In [107]:
alpha_x_1e4 = 0
def anova_plot(alpha_x_1e4, min_fr=0):# for locations: total, amy, hip, 
    #print(infomat.shape)
    do_only=(infomat[:,13] > min_fr)
    infomat_=infomat[do_only, :]
    print(infomat_.shape)
    infomat = infomat_
    alpha = alpha_x_1e4/1e4
    column_labels = ['total', 'amygdala', 'hippocampus', 'EC', 'PHC']
    alpha = .05
    n_units = np.zeros(5)
    n_sign  = np.zeros((3, 5))
    total_reg = sum(infomat[:,9]>0) #total units in valid regions
    n_units[0]  = total_reg
    #  np.logical_and => AttributeError: 'module' object has no attribute 'logial_and'
    n_sign[0,0] = (sum(infomat[j,9]>0 and p_vals[j,4]<alpha for j in range(len(p_vals))))/n_units[0]
    n_sign[1,0] = (sum(infomat[j,9]>0 and p_vals[j,5]<alpha for j in range(len(p_vals))))/n_units[0]
    n_sign[2,0] = (sum(infomat[j,9]>0 and p_vals[j,6]<alpha for j in range(len(p_vals))))/n_units[0]

    for i in range(4): # locations 1-4, beginning in column2 after 'total'
        n_units[i+1]  = sum(infomat[:,9] == i+1)
        n_sign[0,i+1] = (sum(infomat[j,9]==i+1 and p_vals[j,4]<alpha for j in range(len(p_vals))))/n_units[i+1]
        n_sign[1,i+1] = (sum(infomat[j,9]==i+1 and p_vals[j,5]<alpha for j in range(len(p_vals))))/n_units[i+1]
        n_sign[2,i+1] = (sum(infomat[j,9]==i+1 and p_vals[j,6]<alpha for j in range(len(p_vals))))/n_units[i+1]
    print([n_sign]) 

    # prep data for bar plot
    n_units = np.zeros(5)
    bar_widths = np.zeros(5)
    width_factor = .5
    bar_widths[0]=1*width_factor
    n_sign  = np.zeros((3, 5))
    total_reg = sum(infomat[:,9]>0) #total units in valid regions
    n_units[0]  = total_reg
    #  np.logical_and => AttributeError: 'module' object has no attribute 'logial_and'
    n_sign[0,0] = (sum(infomat[j,9]>0 and p_vals[j,4]<alpha for j in range(len(p_vals))))/n_units[0]
    n_sign[1,0] = (sum(infomat[j,9]>0 and p_vals[j,5]<alpha for j in range(len(p_vals))))/n_units[0]
    n_sign[2,0] = (sum(infomat[j,9]>0 and p_vals[j,6]<alpha for j in range(len(p_vals))))/n_units[0]

    for i in range(4): # locations 1-4, beginning in column2 after 'total'
        n_units[i+1]  = sum(infomat[:,9] == i+1)
        bar_widths[i+1]= n_units[i+1]/n_units[0]*width_factor
        print([n_units[i+1],n_units[0], n_units[i+1]/n_units[0]])
        n_sign[0,i+1] = (sum(infomat[j,9]==i+1 and p_vals[j,4]<alpha for j in range(len(p_vals))))/n_units[i+1]
        n_sign[1,i+1] = (sum(infomat[j,9]==i+1 and p_vals[j,5]<alpha for j in range(len(p_vals))))/n_units[i+1]
        n_sign[2,i+1] = (sum(infomat[j,9]==i+1 and p_vals[j,6]<alpha for j in range(len(p_vals))))/n_units[i+1]
    #print(bar_widths) 

    # create bar plot

    init_notebook_mode(connected=True)
    col_names = ['total', 'amygdala', 'hippocampus', 'EC', 'PHC']
    col_names = [1, 3, 4, 5, 6]
    cf.go_offline()
    plotly.offline.init_notebook_mode()
    #n_sign_df = pd.DataFrame(n_sign, columns = col_names)
    #print(bar_widths)
    trace1 = plotly.graph_objs.Bar(x = col_names, y = n_sign[0,:], name = 'main effect memory', width = bar_widths,
                                   offset = -1.5*bar_widths)
    trace2 = plotly.graph_objs.Bar(x = col_names, y = n_sign[1,:], name = 'main effect stimulus', width = bar_widths)
    trace3 = plotly.graph_objs.Bar(x = col_names, y = n_sign[2,:], name = 'interaction', width = bar_widths,
                                  offset = 0.5*bar_widths)
    bar_data = [trace1, trace2, trace3]
    title_text = 'total n='+str(total_reg)
    layout = plotly.graph_objs.Layout(barmode='group',title = 'total n = '+str(total_reg), xaxis=dict(title=
         '                 total                                      amygdala        hippocampus            EC                 PHC'))
    fig = plotly.graph_objs.Figure(data = bar_data, layout = layout)
    #plot(fig, filename='groupedBar')
    iplot(fig, filename='groupedBar')

In [108]:
interact(anova_plot, alpha_x_1e4=1)

A Jupyter Widget

<function __main__.anova_plot>